In [1]:
import numpy as np
# from IPython.display import clear_output
sum_standard = sum
print(sum_standard)

<built-in function sum>


In [2]:

A = [[1, 1, 2, 2], [2, 1, 1, 2], [2, 2, 1, 1], [1, 2, 2, 1]]
B = [[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1], [1, 2, 2, 1]]
_B = [[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1], [1, 1, 2, 0]]
C = np.array([
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 1, 0, 0]], dtype=np.int8)
D = np.array([
            [1, 1, 0, 0], 
            [0, 1, 1, 0], 
            [0, 0, 1, 1], 
            [1, 0, 0, 1]], dtype=np.int8)
E = np.array([  # Unique 
            [1, 2, 2, 1], 
            [2, 1, 1, 2], 
            [2, 0, 0, 0], 
            [1, 2, 0, 0]], dtype=np.int8)
F = np.array([  # Search
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 0, 0, 0]], dtype=np.int8)
# F = np.array([  # Passes validation
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1], 
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1]], dtype=np.int8)


GRID = C.copy()
dim = len(GRID)
dim_2 = dim // 2
FLIP = {2 : 1, 1 : 2}


## Replacing numpy with jampy


In [3]:
class Jolver():
    """General solver class.
    Helper class to Generator."""
    def solve(self, grid):
        """Returns an exhasuted grid.
        Assumes valid grid."""
        grid = grid.copy()
        
        while True:
            full_r = where(grid.all(axis=1))[0]
            full_c = where(grid.all(axis=0))[0]

            if self.subsequent(grid, full_r):
                continue
            changed, new = self.subsequent(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue
            
            # try:
            if self.equal_count(grid, full_r):
                continue
            # except TypeError:
            #     print(grid)
            #     print(full_r)
            #     raise TypeError
            changed, new = self.equal_count(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue

            if self.unique(grid, full_r):
                continue
            changed, new = self.unique(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue

            break
        return grid

    def subsequent(self, rows, full, transposed=False):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        changed = False
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        rows[r, i+2] = FLIP[row[i]]
                        changed = True

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        rows[r, i-1] = FLIP[row[i]]
                        changed = True
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    rows[r, i+1] = FLIP[row[i]]
                    changed = True
        if transposed:
            return changed, rows
        return changed

    def equal_count(self, rows, full, transposed=False):
        # print('equal_count')
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        changed = False
        dim_2 = len(rows) // 2
        for r, row in enumerate(rows):
            if r in full:
                continue
            if sum(row == 1) == dim_2:
                rows[r, where(row == 0)[0]] = 2  # TODO: TypeError: jdarray.__setitem__ key must be int or tuple, not <class 'jampy.jdarray'>
                changed = True

            elif sum(row == 2) == dim_2:
                rows[r, where(row == 0)[0]] = 1
                changed = True
        
        if transposed:
            return changed, rows
        return changed

    def unique(self, rows, full, transposed=False):
        # print('unique')
        """Checks for unique rows and cols.
        Returns True if any changes are made."""
        changed = False

        # Find rows containing exacty two gaps
        gapped_rows_indx = where(sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        # print(full)
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = where(candy == 0)[0]  # [2, 3]
                    print('old', row)
                    rows[gapped_rows_indx[i], gaps] = FLIP[row[gaps]]
                    print('new', rows)
                    changed = True
        if transposed:
            return changed, rows
        return changed

    def get_next(self, grid):
        """Returns the next grid in the sequence.
        Assumes valid grid."""
        grid = grid.copy()

        full_r = where(grid.all(axis=1))[0]
        full_c = where(grid.all(axis=0))[0]

        if action := self._subsequent(grid, full_r):
            return action
        if action := self._subsequent(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._equal_count(grid, full_r):
            return action
        if action := self._equal_count(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._unique(grid, full_r):
            return action
        if action := self._unique(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        raise Exception("No action found")
        
    def _subsequent(self, rows, full):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        return (r, i+2), FLIP[row[i]]

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        return (r, i-1), FLIP[row[i]]
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    return (r, i+1), FLIP[row[i]]

    def _equal_count(self, rows, full):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        dim_2 = len(rows) // 2

        for r, row in enumerate(rows):
            if r in full:
                continue

            if sum(row == 1) == dim_2:
                return (r, where(row == 0)[0][0]), 2

            elif sum(row == 2) == dim_2:
                return (r, where(row == 0)[0][0]), 1
        
    def _unique(self, rows, full):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""

        # Find rows containing exacty two gaps
        gapped_rows_indx = where(sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = where(candy == 0)[0][0]  # [2, 3]
                    return (gapped_rows_indx[i], gaps), FLIP[row[gaps]]


In [4]:
FLIP = np.array([0, 2, 1])
_GRID = np.array([[2, 1, 0, 0],
        [2, 1, 0, 0],
        [1, 2, 1, 2],
        [1, 2, 0, 0]])
_row = np.array([1, 2, 1, 2])
_gaps = np.array([2, 3])
_gapped_rows_indx = 3
print(_row[_gaps])
print(FLIP[_row[_gaps]])
_GRID[_gapped_rows_indx, _gaps] = np.array([2, 1])
# _GRID[_gapped_rows_indx, _gaps] = array([2, 1])
# _GRID[_gapped_rows_indx, _gaps] = FLIP[_row[_gaps]] # array([2, 1])
_GRID 


[1 2]
[2 1]


array([[2, 1, 0, 0],
       [2, 1, 0, 0],
       [1, 2, 1, 2],
       [1, 2, 2, 1]])

In [41]:
FLIP = array([0, 2, 1])
_GRID = array([[2, 1, 0, 0],
        [2, 1, 0, 0],
        [1, 2, 1, 2],
        [1, 2, 0, 0]])
# _row = array([1, 2, 1, 2])
_gaps = array([2, 3])
_gapped_rows_indx = 3
# print(_row[_gaps])
# print(FLIP[_row[_gaps]])
_GRID[_gapped_rows_indx, _gaps] = array([2, 1])
# _GRID[_gapped_rows_indx, _gaps] = FLIP[_row[_gaps]] # array([2, 1])
# _GRID 


array([1, 2, 2, 0])
array([1, 2, 2, 1])
[[2, 1, 0, 0]
 [2, 1, 0, 0]
 [1, 2, 1, 2]
 [1, 2, 2, 1]]


In [43]:
int8=int
# generator = Jenerator()
solver = Jolver()
FLIP = array([0, 2, 1])
_GRID = array([[2, 2, 1, 1],
            [1, 1, 2, 2],
            [0, 0, 1, 0],
            [0, 0, 2, 0]])
solver.solve(_GRID)
# generator.solve_search(_GRID)

old array([1, 2, 1, 2])
array([1, 2, 2, 0])
array([1, 2, 2, 1])
[[2, 1, 0, 0]
 [2, 1, 0, 0]
 [1, 2, 1, 2]
 [1, 2, 2, 1]]
new [[2, 1, 0, 0]
 [2, 1, 0, 0]
 [1, 2, 1, 2]
 [1, 2, 2, 1]]


[[2, 2, 1, 1]
 [1, 1, 2, 2]
 [0, 0, 1, 2]
 [0, 0, 2, 1]]

In [42]:
"""Script replecating numpy for use in pygbag that only takes native libraries."""
# int8 = int
# sum_standard = sum

def array(array):
    return jdarray(array)

def ones(shape, dtype=None):
    array = jdarray(shape)
    array._build(shape, 1)
    return array

def zeros(shape, dtype=None):
    array = jdarray(shape)
    array._build(shape, 0)
    return array

def all(array):
    if isinstance(array[0], list):  # 2d
        for row in array:
            for item in row:
                if not item:
                    return False
    else:  # 1d
        for item in array:
            if not item:
                return False
    return True

def any(array):
    if isinstance(array[0], list):  # 2d
        for row in array:
            for item in row:
                if item:
                    return True
    else:  # 1d
        for item in array:
            if item:
                return True
    return False

def sum(array, axis=None):
    return array.sum(axis)

def _where(array):
    return [(i, j) for i, row in enumerate(array) for j, item in enumerate(row) if item]

def where(array):
    """ Returns two lists of indices of non-zero elements.
    Use _where instead to get a list of tuples."""
    if isinstance(array[0], int):  # 1d
        return jdarray([i for i, item in enumerate(array) if item]), None # dtype from np
    
    rows, cols = [], []
    for i, row in enumerate(array):
        for j, item in enumerate(row):
            if item:
                rows.append(i)
                cols.append(j)
    return jdarray(rows), jdarray(cols)

def nonzero(array):
    return where(array)

def unique(array, axis):
    """Returns a list of unique rows or columns"""
    if axis == 0:
        return jdarray([array[i] for i in range(len(array)) if array[i] not in array[:i]])
    elif axis == 1:
        raise NotImplementedError("unique axis 1 not implemented, due to numpy's being weird")
    else:
        raise ValueError(f"unique axis must be 0 or 1, not {axis}")

def array_equal(array1, array2): # TODO: Test
    if isinstance(array1[0], list):  # 2d
        for i, row in enumerate(array1):
            for j, item in enumerate(row):
                if item != array2[i][j]:
                    return False
    else:  # 1d
        for i, item in enumerate(array1):
            if item != array2[i]:
                return False
    return True

class jdarray():
    """A ndarray clone that supports only 2d arrays, but a few places are written to support more dimensions.
    1d array is implemented too, for list > int stuff"""

    def __init__(self, shape):
        """If shape, remember to call _ones or _zeros. Otherwise giving a list works fine."""
        if isinstance(shape, int):
            self.shape = (shape,)
        elif isinstance(shape, tuple):
            self.shape = shape

        elif isinstance(shape, list):  # Broken 'shape' name
            array = shape
            if array == []:
                self.shape = (0,)
                self.array = []
            elif isinstance(array[0], list):
                self.shape = (len(array), len(array[0]))
                self.array = [jdarray(row) for row in array]
            elif isinstance(array[0], tuple):  # Weird, but used when defining tiles to be shuffled [(0, 0), (0, 1), ...]
                self.shape = (len(array), len(array[0]))
                self.array = [jdarray(list(row)) for row in array]

            elif isinstance(array, list):  # 1d
                self.shape = (len(array), )
                self.array = array
            else:
                raise TypeError(f"jdarray.__init__ shape must be int, tuple or list, not {type(shape)}")
        else:
            raise TypeError(f"jdarray.__init__ shape must be int, tuple or list, not {type(shape)}")
    
    def copy(self):
        # return jdarray([list(row) for row in self.array])
        copy = jdarray(self.shape)
        copy.array = [row for row in self.array]
        return copy
    
    def sum(self, axis=None):
        if axis is None:  
            if isinstance(self.array[0], int):
                return sum_standard(self.array) # 1d
            return sum_standard([sum_standard(row) for row in self.array])
        elif axis == 0:
            # return [sum_standard(row) for row in zip(*self.array)]
            return jdarray([sum_standard(row) for row in zip(*self.array)])
        elif axis == 1:
            # return [sum_standard(row) for row in self.array]
            return jdarray([sum_standard(row) for row in self.array])
    
    def _build(self, shape, value=0):
        if isinstance(shape, int):
            self.array = [value] * shape

        elif len(shape) == 1:
            return jdarray([value] * shape[0])
        else:
            self.array = [self._build(shape[1:], value) for _ in range(shape[0])]
    
    @property
    def T(self):
        return jdarray([list(row) for row in zip(*self.array)])
    
    def all(self, axis=None):
        if axis is None:
            return all(self.array)
        
        elif axis == 0:
            return jdarray([all(row) for row in zip(*self.array)])
        elif axis == 1:
            return jdarray([all(row) for row in self.array])


    def __getitem__(self, key):
        if isinstance(key, int):
            return self.array[key]
        
        elif isinstance(key, tuple):
            # print('tuple')
            # if isinstance(key[0], jdarray):
            #     if len(key) > 2:
            #         raise IndexError(f"jdarray.__getitem__ key {key} is out of range")
            #     return jdarray([self.array[i] for i in key[0].array])[key[1]]

            if len(key) > len(self.shape):
                raise IndexError(f"jdarray.__getitem__ key {key} is out of range")
            return self.array[key[0]][key[1]]

        elif isinstance(key, slice):
            # print('slice')
            return jdarray(self.array[key])
        
        elif isinstance(key, jdarray):
            # print('jdarray')
            if len(key.shape) == 1:
                return jdarray([self.array[i] for i in key.array])
            return [self.array[i][j] for i, row in enumerate(key.array) for j, b in enumerate(row) if b]

        else:
            raise TypeError(f"jdarray.__getitem__ key must be int or tuple, not {type(key)}")
    
    def __setitem__(self, key, value):
        # print('SET_ITEM')
        # print(type(key), type(value))
        # print(key)
        # print(value)
        assert(type(self[key]) == type(value)), f"jdarray.__setitem__ key {key} is {type(self[key])}, but value is {type(value)}"
        if isinstance(key, int):  # 1d
            # print('int')
            self.array[key] = value

        elif isinstance(key, tuple):
            # print('tuple')
            if len(key) > len(self.shape):
                raise IndexError(f"jdarray.__getitem__ key {key} is has too many dimensions")
            
            if isinstance(key[0], jdarray):
                # print('jdarray [0]')
                if isinstance(key[1], jdarray):
                    # print('jdarray [0] jdarray value')
                    for k, v in zip(key[0], value):
                        self.array[k][key[1]] = v
                else:
                    for k in key[0]:
                        self.array[k][key[1]] = value

            elif isinstance(key[1], jdarray):
                if isinstance(value, jdarray):
                    # print('jdarray [1] jdarray value')
                    for k, v in zip(key[1], value):
                        self.array[key[0]][k] = v
                else:
                    # print('jdarray [1]')
                    for k in key[1]:
                        self.array[key[0]][k] = value
            else:
                self.array[key[0]][key[1]] = value
        # else:
        #     raise TypeError(f"jdarray.__setitem__ key must be int or tuple, not {key, type(key)}")
        print(self)
        print('====================')
        
    def __len__(self):
        return len(self.array)
    
    def __iter__(self):
        return iter(self.array)
    
    def __contains__(self, item):
        if self.array == []:
            return False
        if isinstance(self.array[0], int):  # 1d
            return item in self.array
        
        # 2d
        if isinstance(item, int):
            for row in self.array:
                if item in row:
                    return True
        else:
            for row in self.array:
                if item == row:
                    return True
        return False
    
    def __hash__(self):
        return hash(tuple([item for row in self.array for item in row]))
    
    def __mul__(self, integer):
        if isinstance(integer, int):
            result = jdarray(self.shape)
            result.array = self._mul(self.array, integer)
            return result
        else:
            raise TypeError(f"jdarray.__mul__ integer must be int, not {type(integer)}")
    
    def _mul(self, array, integer):
        if isinstance(array[0], int):  # No more nests
            return [integer * element for element in array]
        return [jdarray(self._mul(entree, integer)) for entree in array]
    
    def __floordiv__(self, integer):
        if isinstance(integer, int):
            result = jdarray(self.shape)
            result.array = self._floordiv(self.array, integer)
            return result
        else:
            raise TypeError(f"jdarray.__floordiv__ integer must be int, not {type(integer)}")
    
    def _floordiv(self, array, integer):
        if isinstance(array[0], int):
            return [element // integer for element in array]
        return [jdarray(self._floordiv(entree, integer)) for entree in array]
    
    def __repr__(self):
        if len(self.shape) == 1:
            return f'array({str(self.array)})'
        string = "["
        for row in self.array:
            string += str(row.array) + '\n '
        return string[:-2] + "]"
    
    def __add__(self, other):
        if isinstance(other, int):
            result = jdarray(self.shape)
            result.array = self._add(self.array, other)
            return result
        else:
            raise TypeError(f"jdarray.__add__ other must be int, not {type(other)}")
    
    def _add(self, array, other):
        if isinstance(array[0], int):
            return [element + other for element in array]
        return [self._add(entree, other) for entree in array]

    def __eq__(self, other):
        """Double function, checks if equal array or returns bool list of equalities to int"""
        if isinstance(other, int): # Get bool list of equalities
            result = jdarray(self.shape)
            result.array = self._eq(self.array, other)
            return result
        elif isinstance(other, jdarray):  # Check if equal
            return self.array == other.array
        else:
            raise TypeError(f"jdarray.__eq__ other must be int, not {type(other)}")
    
    def _eq(self, array, other):
        assert(not isinstance(array, np.int32)), "array is np.int32"
        # print("type", type(array))
        # print("array", array)
        if isinstance(array[0], int):
            return [element == other for element in array]
        return jdarray([self._eq(entree, other) for entree in array])

    def __ne__(self, other):
        if isinstance(other, int):
            array = self._ne(self.array, other)
            result = jdarray(self.shape)
            result.array = array
            return result
        else:
            raise TypeError(f"jdarray.__ne__ other must be int, not {type(other)}")
    
    def _ne(self, array, other):
        if isinstance(array[0], int):
            return [element != other for element in array]
        return [self._ne(entree, other) for entree in array]
    
    def __gt__(self, other):  # 
        if isinstance(other, int):
            array = self._gt(self.array, other)
            result = jdarray(self.shape)
            result.array = array
            return result
        else:
            raise TypeError(f"jdarray.__gt__ other must be int, not {type(other)}")
    
    def _gt(self, array, other):
        if isinstance(array[0], int):
            return [element > other for element in array]
        return [self._gt(entree, other) for entree in array]
    
    def __lt__(self, other):
        if isinstance(other, int):
            array = self._lt(self.array, other)
            result = jdarray(self.shape)
            result.array = array
            return result
        else:
            raise TypeError(f"jdarray.__lt__ other must be int, not {type(other)}")
        
    def _lt(self, array, other):
        if isinstance(array[0], int):
            return [element < other for element in array]
        return [self._lt(entree, other) for entree in array]

from random import randint, shuffle, seed

class Random():
    
    def randint(self, lower, upper=0):
        if upper == 0:
            return randint(0, lower-1)
        return randint(lower, upper-1)
    
    def shuffle(self, array):
        shuffle(array)
        return array
    
    def seed(self, s):
        seed(s)
        
random = Random()

In [4]:
_row = array([0, 2, 1, 1])
type(_row)
# _row = [0, 2, 1, 1]
_row == 1

array([False, False, True, True])

### Testing

In [5]:
# Instancing
print("Instancing")
_shape = (2, 3)
narray = np.zeros(_shape)
narray[1,1] = 5
print(narray)

jarray = zeros(_shape)
jarray[1, 1] = 5
print(jarray)

print("\nMultiplication")
jarray = ones(_shape)
print(jarray, 'before')
jarray *= 4
print(jarray, 'after')

print("\nSumming")
# AttributeError: 'list' object has no attribute 'sum' - standard_sum was overwritten
# TypeError: unsupported operand type(s) for +: 'int' and 'list' - axis=None for 2d wasn't implemented
_shape = (2, 3)
narray = np.ones(_shape)  
jarray = ones(_shape)  
# jarray.sum()
# print(f'Total: numpy: {narray.sum()}\t Jarray: {jarray.sum()}')  
print(f'Rows : numpy: {narray.sum(axis=1)}\t Jarray: {jarray.sum(axis=1)}')
print(f'Cols : numpy: {narray.sum(axis=0)}\t Jarray: {jarray.sum(axis=0)}')

print('\nEqual')
_shape = (2, 3)
narray = np.ones(_shape)
jarray = ones(_shape)
narray[1,1] = 0
jarray[1,1] = 0
print(narray[narray == 1])
print(jarray[jarray == 1])

print('\nWhere (1d then 2d on axis 1)')
GRID = np.array(A)
print(np.where(GRID == 1))
print(list(zip(*np.where(GRID == 1))))
GJID = array(A)
print(GJID)
print(where(GJID == 1))  # TypeError: 'bool' object is not iterable # Broken for 1d lists
print(list(zip(*where(GJID == 1))))

print()
_A = A.copy()
_A[2][1] = 0
GRID = np.array(_A)
GJID = array(A)
print(np.where(GRID.all(axis=1))[0])
print(where(GJID.all(axis=1))[0])


print('\nUnique rows and then columns')
GRID = np.array(B)
GJID = array(B)
print(np.unique(GRID, axis=0))
print(unique(GJID, axis=0))
print()
GRID = np.array(_B).T
GJID = array(_B).T
print(np.unique(GRID, axis=0))
print(unique(GJID, axis=0))

print('\nShuffle')
tiles = [(r, c) for r in range(2) for c in range(2)]
GRID = np.array(tiles)
np.random.shuffle(GRID)
print(GRID)

GRID = jdarray(tiles)
random.shuffle(GRID)
print(GRID)




Instancing
[[0. 0. 0.]
 [0. 5. 0.]]
[[0, 0, 0]
 [0, 5, 0]]

Multiplication
[[1, 1, 1]
 [1, 1, 1]] before
[[4, 4, 4]
 [4, 4, 4]] after

Summing
Rows : numpy: [3. 3.]	 Jarray: array([3, 3])
Cols : numpy: [2. 2. 2.]	 Jarray: array([2, 2, 2])

Equal
[1. 1. 1. 1. 1.]
[1, 1, 1, 1, 1]

Where (1d then 2d on axis 1)
(array([0, 0, 1, 1, 2, 2, 3, 3], dtype=int64), array([0, 1, 1, 2, 2, 3, 0, 3], dtype=int64))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]
[[1, 1, 2, 2]
 [2, 1, 1, 2]
 [2, 2, 1, 1]
 [1, 2, 2, 1]]
(array([0, 0, 1, 1, 2, 2, 3, 3]), array([0, 1, 1, 2, 2, 3, 0, 3]))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]

[0 1 3]
array([0, 1, 3])

Unique rows and then columns
[[1 1 2 2]
 [1 2 2 1]
 [2 2 1 1]]
array([array([1, 1, 2, 2]), array([2, 2, 1, 1]), array([1, 2, 2, 1])])

[[1 2 2 1]
 [2 1 1 0]
 [2 1 1 2]]
array([array([1, 2, 2, 1]), array([2, 1, 1, 2]), array([2, 1, 1, 0])])

Shuffle
[[0 0]
 [1 1]
 [1 0]
 [0 1]]
[[1, 1]
 [0, 1]
 [0, 0]
 [1, 0]]


In [6]:
GRID = np.array(A)
print(GRID == 1)
print(np.sum(GRID == 1, axis=0))
print(np.sum(GRID == 1, axis=0) > dim_2)
print(np.any(np.sum(GRID == 1, axis=0) > dim_2))
GJID = array(A)
print(GJID == 1)
print(sum(GJID == 1, axis=0))
print(sum(GJID == 1, axis=0) > dim_2)
print(any(sum(GJID == 1, axis=0) > dim_2))

[[ True  True False False]
 [False  True  True False]
 [False False  True  True]
 [ True False False  True]]
[2 2 2 2]
[False False False False]
False
[[True, True, False, False]
 [False, True, True, False]
 [False, False, True, True]
 [True, False, False, True]]
array([2, 2, 2, 2])
array([False, False, False, False])
False


In [7]:
GRID = np.array(A)
print(np.where(GRID == 1))
print(list(zip(*np.where(GRID == 1))))

GJID = array(A)
print(where(GJID == 1))
print(list(zip(*where(GJID == 1))))

(array([0, 0, 1, 1, 2, 2, 3, 3], dtype=int64), array([0, 1, 1, 2, 2, 3, 0, 3], dtype=int64))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]
(array([0, 0, 1, 1, 2, 2, 3, 3]), array([0, 1, 1, 2, 2, 3, 0, 3]))
[(0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 3), (3, 0), (3, 3)]


## Rules

In [8]:
def valid_grid(grid, verbose=False):
    """Check that grid is valid."""
    dim = len(grid)
    dim_2 = dim // 2
    
    # Equal count
    if np.any(np.sum(grid == 1, axis=1) > dim_2) or np.any(np.sum(grid == 2, axis=1) > dim_2):
        if verbose:
            print("Unequal count - rows")
        return False
    if np.any(np.sum(grid == 1, axis=0) > dim_2) or np.any(np.sum(grid == 2, axis=0) > dim_2):
        if verbose:
            print("Unequal count - cols")
        return False
    
    # Subsequent (3 in a row)
    for r in range(dim):
        for c in range(dim):
            if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                if verbose:
                    print("Subsequent - rows")
                return False
            if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                if verbose:
                    print("Subsequent - cols")
                return False
    
    # Unique
    full_rows = grid[np.where(grid.all(axis=1))[0]]
    if len(full_rows) != len(np.unique(full_rows, axis=0)):
        if verbose:
            print("Duplicate - rows")
        return False
    
    full_cols = grid.T[np.where(grid.all(axis=0))[0]]
    if len(full_cols) != len(np.unique(full_cols, axis=0)):
        if verbose:
            print("Duplicate - cols")
        return False
    
    return True

def jalid_grid(grid, verbose=False):
    """Check that grid is valid."""
    dim = len(grid)
    dim_2 = dim // 2
    
    # Equal count
    if any(sum(grid == 1, axis=1) > dim_2) or any(sum(grid == 2, axis=1) > dim_2):
        if verbose:
            print("Unequal count - rows")
        return False
    if any(sum(grid == 1, axis=0) > dim_2) or any(sum(grid == 2, axis=0) > dim_2):
        if verbose:
            print("Unequal count - cols")
        return False
    
    # Subsequent (3 in a row)
    for r in range(dim):
        for c in range(dim):
            if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                if verbose:
                    print("Subsequent - rows")
                return False
            if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                if verbose:
                    print("Subsequent - cols")
                return False
    
    # Unique
    full_rows = grid[where(grid.all(axis=1))[0]]
    if len(full_rows) != len(unique(full_rows, axis=0)):
        if verbose:
            print("Duplicate - rows")
        return False
    
    full_cols = grid.T[where(grid.all(axis=0))[0]]
    if len(full_cols) != len(unique(full_cols, axis=0)):
        if verbose:
            print("Duplicate - cols")
        return False
    
    return True

## Solver

In [37]:
class Jolver():
    """General solver class.
    Helper class to Generator."""
    def solve(self, grid):
        """Returns an exhasuted grid.
        Assumes valid grid."""
        grid = grid.copy()
        
        while True:
            full_r = where(grid.all(axis=1))[0]
            full_c = where(grid.all(axis=0))[0]

            if self.subsequent(grid, full_r):
                continue
            changed, new = self.subsequent(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue
            
            try:
                if self.equal_count(grid, full_r):
                    continue
            except TypeError:
                print(grid)
                print(full_r)
                raise TypeError
            changed, new = self.equal_count(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue

            if self.unique(grid, full_r):
                continue
            changed, new = self.unique(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue

            break
        return grid

    def subsequent(self, rows, full, transposed=False):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        changed = False
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        rows[r, i+2] = FLIP[row[i]]
                        changed = True

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        rows[r, i-1] = FLIP[row[i]]
                        changed = True
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    rows[r, i+1] = FLIP[row[i]]
                    changed = True
        if transposed:
            return changed, rows
        return changed

    def equal_count(self, rows, full, transposed=False):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        changed = False
        dim_2 = len(rows) // 2
        for r, row in enumerate(rows):
            if r in full:
                continue
            if sum(row == 1) == dim_2:
                rows[r, where(row == 0)[0]] = 2  # TODO: TypeError: jdarray.__setitem__ key must be int or tuple, not <class 'jampy.jdarray'>
                changed = True

            elif sum(row == 2) == dim_2:
                rows[r, where(row == 0)[0]] = 1
                changed = True
        
        if transposed:
            return changed, rows
        return changed

    def unique(self, rows, full, transposed=False):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""
        changed = False

        # Find rows containing exacty two gaps
        gapped_rows_indx = where(sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        # print(full)
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = where(candy == 0)[0]  # [2, 3]
                    rows[gapped_rows_indx[i], gaps] = FLIP[row[gaps]]
                    changed = True
        
        if transposed:
            return changed, rows
        return changed

    def get_next(self, grid):
        """Returns the next grid in the sequence.
        Assumes valid grid."""
        grid = grid.copy()

        full_r = where(grid.all(axis=1))[0]
        full_c = where(grid.all(axis=0))[0]

        if action := self._subsequent(grid, full_r):
            return action
        if action := self._subsequent(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._equal_count(grid, full_r):
            return action
        if action := self._equal_count(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._unique(grid, full_r):
            return action
        if action := self._unique(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        raise Exception("No action found")
        
    def _subsequent(self, rows, full):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        return (r, i+2), FLIP[row[i]]

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        return (r, i-1), FLIP[row[i]]
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    return (r, i+1), FLIP[row[i]]

    def _equal_count(self, rows, full):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        dim_2 = len(rows) // 2

        for r, row in enumerate(rows):
            if r in full:
                continue

            if sum(row == 1) == dim_2:
                return (r, where(row == 0)[0][0]), 2

            elif sum(row == 2) == dim_2:
                return (r, where(row == 0)[0][0]), 1
        
    def _unique(self, rows, full):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""

        # Find rows containing exacty two gaps
        gapped_rows_indx = where(sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = where(candy == 0)[0][0]  # [2, 3]
                    return (gapped_rows_indx[i], gaps), FLIP[row[gaps]]

class Solver():
    """General solver class.
    Helper class to Generator."""
    
    def solve(self, grid):
        """Returns an exhasuted grid.
        Assumes valid grid."""
        grid = grid.copy()
        
        while True:
            full_r = np.where(grid.all(axis=1))[0]
            full_c = np.where(grid.all(axis=0))[0]

            if self.subsequent(grid, full_r):
                continue
            changed, new = self.subsequent(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue

            if self.equal_count(grid, full_r):
                continue
            changed, new = self.equal_count(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue

            if self.unique(grid, full_r):
                continue
            changed, new = self.unique(grid.T, full_c, True)
            if changed:
                grid = new.T
                continue
            break
        return grid

    def subsequent(self, rows, full, transposed=False):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        changed = False
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        rows[r, i+2] = FLIP[row[i]]
                        changed = True

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        rows[r, i-1] = FLIP[row[i]]
                        changed = True
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    rows[r, i+1] = FLIP[row[i]]
                    changed = True

        if transposed:
            return changed, rows
        return changed

    def equal_count(self, rows, full, transposed=False):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        changed = False
        dim_2 = len(rows) // 2
        for r, row in enumerate(rows):
            if r in full:
                continue
            if np.sum(row == 1) == dim_2:
                rows[r, np.where(row == 0)[0]] = 2  # TODO: TypeError: jdarray.__setitem__ key must be int or tuple, not <class 'jampy.jdarray'>
                changed = True

            elif np.sum(row == 2) == dim_2:
                rows[r, np.where(row == 0)[0]] = 1
                changed = True
        
        if transposed:
            return changed, rows
        return changed

    def unique(self, rows, full, transposed=False):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""
        changed = False

        # Find rows containing exacty two gaps
        gapped_rows_indx = np.where(np.sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        # print(full)
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = np.where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if np.array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = np.where(candy == 0)[0]  # [2, 3]
                    rows[gapped_rows_indx[i], gaps] = FLIP[row[gaps]]

                    changed = True
        
        if transposed:
            return changed, rows
        return changed

    def get_next(self, grid):
        """Returns the next grid in the sequence.
        Assumes valid grid."""
        grid = grid.copy()

        full_r = np.where(grid.all(axis=1))[0]
        full_c = np.where(grid.all(axis=0))[0]

        if action := self._subsequent(grid, full_r):
            return action
        if action := self._subsequent(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._equal_count(grid, full_r):
            return action
        if action := self._equal_count(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._unique(grid, full_r):
            return action
        if action := self._unique(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        raise Exception("No action found")
        
    def _subsequent(self, rows, full):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        return (r, i+2), FLIP[row[i]]

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        return (r, i-1), FLIP[row[i]]
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    return (r, i+1), FLIP[row[i]]

    def _equal_count(self, rows, full):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        dim_2 = len(rows) // 2

        for r, row in enumerate(rows):
            if r in full:
                continue

            if np.sum(row == 1) == dim_2:
                return (r, np.where(row == 0)[0][0]), 2

            elif np.sum(row == 2) == dim_2:
                return (r, np.where(row == 0)[0][0]), 1
        
    def _unique(self, rows, full):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""

        # Find rows containing exacty two gaps
        gapped_rows_indx = np.where(np.sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = np.where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if np.array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = np.where(candy == 0)[0][0]  # [2, 3]
                    return (gapped_rows_indx[i], gaps), FLIP[row[gaps]]




In [ ]:
_GRID = [[0, 0, 1, 1],
            [1, 2, 0, 1],
            [0, 0, 0, 0],
            [2, 0, 0, 0]]

GRID = np.array(_GRID)
solver = Solver()
print(solver.solve(GRID))

GRID = array(_GRID)
jolver = Jolver()
print(jolver.solve(GRID))



In [11]:
def subsequent(rows, full, transposed=False):
    """Checks for subsequent 1s and 2s in rows and cols.
    Returns True if any changes are made."""
    changed = False
    dim = len(rows)

    for r, row in enumerate(rows):
        if r in full:
            continue
        
        for i in range(len(row)-1):
            if row[i] == row[i+1] != 0:
                if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                    rows[r, i+2] = FLIP[row[i]]
                    changed = True

                if i > 0 and rows[r, i-1] == 0: # if prev is empty
                    rows[r, i-1] = FLIP[row[i]]
                    changed = True
                
            if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                rows[r, i+1] = FLIP[row[i]]
                changed = True

    return changed if not transposed else changed, rows.T


def equal_count(rows, full, transposed=False):
    """Checks for equal count of 1s and 2s.
    param rows: 2D array
    param full: set of rows or cols that are already full
    Returns True if any changes are made."""
    changed = False
    dim_2 = len(rows) // 2
    
    for r, row in enumerate(rows):
        if r in full:
            print('full')
            continue
        # continue
        try:
            sum(row == 1)
        except AssertionError:
            print('AssertionError')
            print(type(row[0]))
            print()
            continue
            
        if sum(row == 1) == dim_2:
            rows[r, where(row == 0)[0]] = 2  # TODO: TypeError: jdarray.__setitem__ key must be int or tuple, not <class 'jampy.jdarray'>
            changed = True

        elif sum(row == 2) == dim_2:
            print('HIT:', r, row)
            print(row == 0)
            print(where(row == 0))
            print(where(row == 0)[0])
            print('before', rows[r, where(row == 0)[0]])
            print(row)
            rows[r, where(row == 0)[0]] = 1
            print('after', rows[r, where(row == 0)[0]])
            print(row)
            changed = True
    
    return changed if not transposed else changed, rows.T


solver = Solver()
_GRID = [[0, 0, 1, 1],
            [1, 0, 0, 0],
            [0, 0, 0, 0],
            [2, 0, 0, 0]]
GRID = np.array(_GRID)
# print(solver.solve(GRID))

GRID = jdarray(_GRID)
jolver = Jolver()
# print(jolver.solve(GRID))
full_r = where(GRID.all(axis=1))[0]
full_c = where(GRID.all(axis=0))[0]
subsequent(GRID, full_r)
print(GRID)
# equal_count(GRID, full_r)
# full_r = where(GRID.all(axis=1))[0]
# _, GRID = equal_count(GRID.T, full_c, transposed=True)
# print(GRID)
# equal_count(GRID.T, full_c)
# print(GRID)


[[0, 2, 1, 1]
 [1, 0, 0, 0]
 [0, 0, 0, 0]
 [2, 0, 0, 0]]


In [12]:
GRID = jdarray(_GRID)
subsequent(GRID, full_r)
equal_count(GRID, full_r)

print(GRID)
_rows = GRID.T
_r = 0
_row = array([2, 1, 0, 2])
print(_row == 0)
print(where(_row == 0))
print(where(_row == 0)[0])
print(_rows[_r, where(_row == 0)[0]])
_rows[_r, where(_row == 0)[0]] = 1
print(_rows[_r, where(_row == 0)[0]])
print(GRID)

[[2, 2, 1, 1]
 [1, 0, 0, 0]
 [0, 0, 0, 0]
 [2, 0, 0, 0]]
array([False, False, True, False])
(array([2]), None)
array([2])
array([0])
array([1])
[[2, 2, 1, 1]
 [1, 0, 0, 0]
 [0, 0, 0, 0]
 [2, 0, 0, 0]]


In [13]:
def T(array):
    # Transpose, but return same array
    return list(map(list, zip(*array)))
    return array

GRID = jdarray(_GRID)
print(GRID)
GRID_T = T(GRID)
GRID_T[1][1] = 5
print(GRID)

[[2, 2, 1, 1]
 [1, 0, 0, 0]
 [0, 0, 0, 0]
 [2, 0, 0, 0]]
[[2, 2, 1, 1]
 [1, 0, 0, 0]
 [0, 0, 0, 0]
 [2, 0, 0, 0]]


## Generator

In [39]:
class Generator():
    """Generates Takuzu puzzles."""
    # https://stackoverflow.com/questions/6924216/how-to-generate-sudoku-boards-with-unique-solutions

    def __init__(self):
        self.solver = Solver()
        self.solve = self.solver.solve
    
    def valid_grid(self, grid, verbose=False):
        """Check that grid is valid."""
        dim = len(grid)
        dim_2 = dim // 2
        
        # Equal count
        if np.any(np.sum(grid == 1, axis=1) > dim_2) or np.any(np.sum(grid == 2, axis=1) > dim_2):
            if verbose:
                print("Unequal count - rows")
            return False
        if np.any(np.sum(grid == 1, axis=0) > dim_2) or np.any(np.sum(grid == 2, axis=0) > dim_2):
            if verbose:
                print("Unequal count - cols")
            return False
        
        # Subsequent (3 in a row)
        for r in range(dim):
            for c in range(dim):
                # try:
                if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                    if verbose:
                        print("Subsequent - rows")
                    return False
                if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                    if verbose:
                        print("Subsequent - cols")
                    return False
        
        # Unique
        full_rows = grid[np.where(grid.all(axis=1))[0]]
        if len(full_rows) != len(np.unique(full_rows, axis=0)):
            if verbose:
                print("Duplicate - rows")
            return False
        
        full_cols = grid.T[np.where(grid.all(axis=0))[0]]
        if len(full_cols) != len(np.unique(full_cols, axis=0)):
            if verbose:
                print("Duplicate - cols")
            return False
        
        return True

    def generate_grid(self, dim):
        new = np.zeros((dim, dim), dtype=np.int8)
        # return new
        complete = self.solve_search(new)
        return complete
        grid = self.subtract(complete)
        return grid
    
    def solve_search(self, grid, depth=0):
        """Solves and searches for solution.
        Returns None if no solution found or grid if solution found."""
        # print(depth)
        # print(grid)
        grid = self.solve(grid)
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            return None
        
        if 0 not in grid:  # Solved, so terminal state
            return grid

        # Randomly select empty cell and fill with 1 or 2
        # empty_cells = np._where(grid == 0)
        empty_cells = list(zip(*np.where(grid == 0)))
        tile = empty_cells[np.random.randint(len(empty_cells))]
        color = np.random.randint(1, 3)

        grid[tile] = color
        result = self.solve_search(grid, depth+1)

        if result is None:
            grid[tile] = FLIP[color]
            result = self.solve_search(grid, depth+1)
        return result

    # First algorithm from stackoverflow
    def subtract(self, grid):
        """Subtracts a random tiles until one more subtraction requres solve_search."""
        
        # 2) Shuffle a list of all tiles
        tiles = [(r, c) for r in range(len(grid)) for c in range(len(grid))]
        np.random.shuffle(tiles) 

        while tiles:
            # 3) Remove a tile from grid
            tile = tiles.pop()
            color = grid[tile]
            grid[tile] = 0

            # # 4) Test uniqueness of grid
            # if not self.is_unique(grid):
            #     # 6) If not unique, add tile back and continue
            #     grid[tile] = color

            if 0 in self.solve(grid):
                grid[tile] = color

        return grid

    def is_unique(self, grid):
        """Checks if grid has unique solution."""
        return self.unique_solution(grid) == 1
    
    def unique_solution(self, grid, solved=False):
        """Solves grid.
        Returns False if multiple solutions."""

        grid = self.solve(grid)
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            return False
        
        if 0 not in grid:  # Solved, so terminal state
            return 1

        # Find frist instance of 0
        zeros = np.where(grid == 0)
        tile = zeros[0][0], zeros[1][0]

        # First go left, then right
        grid[tile] = 1
        left = self.unique_solution(grid, solved)

        grid[tile] = 2
        right = self.unique_solution(grid, solved)

        if left == 2 or right == 2:
            return 2
        if left == right == 1:
            return 2
        return self.unique_solution(grid, solved)

class Jenerator():
    """Generates Takuzu puzzles."""
    # https://stackoverflow.com/questions/6924216/how-to-generate-sudoku-boards-with-unique-solutions

    def __init__(self):
        self.solver = Jolver()
        self.solve = self.solver.solve
    
    def valid_grid(self, grid, verbose=False):
        """Check that grid is valid."""
        dim = len(grid)
        dim_2 = dim // 2
        
        # Equal count
        if any(sum(grid == 1, axis=1) > dim_2) or any(sum(grid == 2, axis=1) > dim_2):
            if verbose:
                print("Unequal count - rows")
            return False
        if any(sum(grid == 1, axis=0) > dim_2) or any(sum(grid == 2, axis=0) > dim_2):
            if verbose:
                print("Unequal count - cols")
            return False
        
        # Subsequent (3 in a row)
        for r in range(dim):
            for c in range(dim):
                # try:
                if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                    if verbose:
                        print("Subsequent - rows")
                    return False
                if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                    if verbose:
                        print("Subsequent - cols")
                    return False
        
        # Unique
        full_rows = grid[where(grid.all(axis=1))[0]]
        if len(full_rows) != len(unique(full_rows, axis=0)):
            if verbose:
                print("Duplicate - rows")
            return False
        
        full_cols = grid.T[where(grid.all(axis=0))[0]]
        if len(full_cols) != len(unique(full_cols, axis=0)):
            if verbose:
                print("Duplicate - cols")
            return False
        
        return True

    def generate_grid(self, dim):
        new = zeros((dim, dim), dtype=int8)
        # return new
        complete = self.solve_search(new)
        return complete
        grid = self.subtract(complete)
        return grid
    
    def solve_search(self, grid, depth=0):
        """Solves and searches for solution.
        Returns None if no solution found or grid if solution found."""
        # print(depth)
        # print(grid)
        # if depth == 10:
        #     return None
        # try:
        grid = self.solve(grid)
        # except TypeError:
        #     print(grid)
        #     raise TypeError
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            return None
        
        if 0 not in grid:  # Solved, so terminal state
            return grid

        # Randomly select empty cell and fill with 1 or 2
        # empty_cells = _where(grid == 0)
        empty_cells = list(zip(*where(grid == 0)))
        tile = empty_cells[random.randint(len(empty_cells))]
        color = random.randint(1, 3)

        grid[tile] = color
        result = self.solve_search(grid, depth+1)

        if result is None:
            grid[tile] = FLIP[color]
            result = self.solve_search(grid, depth+1)
        return result

    # First algorithm from stackoverflow
    def subtract(self, grid):
        """Subtracts a random tiles until one more subtraction requres solve_search."""
        
        # 2) Shuffle a list of all tiles
        tiles = [(r, c) for r in range(len(grid)) for c in range(len(grid))]
        random.shuffle(tiles) 

        while tiles:
            # 3) Remove a tile from grid
            tile = tiles.pop()
            color = grid[tile]
            grid[tile] = 0

            # # 4) Test uniqueness of grid
            # if not self.is_unique(grid):
            #     # 6) If not unique, add tile back and continue
            #     grid[tile] = color

            if 0 in self.solve(grid):
                grid[tile] = color

        return grid

    def is_unique(self, grid):
        """Checks if grid has unique solution."""
        return self.unique_solution(grid) == 1
    
    def unique_solution(self, grid, solved=False):
        """Solves grid.
        Returns False if multiple solutions."""

        grid = self.solve(grid)
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            return False
        
        if 0 not in grid:  # Solved, so terminal state
            return 1

        # Find frist instance of 0
        zeros = where(grid == 0)
        tile = zeros[0][0], zeros[1][0]

        # First go left, then right
        grid[tile] = 1
        left = self.unique_solution(grid, solved)

        grid[tile] = 2
        right = self.unique_solution(grid, solved)

        if left == 2 or right == 2:
            return 2
        if left == right == 1:
            return 2
        return self.unique_solution(grid, solved)



In [40]:
int8=int
generator = Jenerator()
solver = Jolver()
FLIP = array([0, 2, 1])
_GRID = array([[2, 2, 1, 1],
            [1, 1, 2, 2],
            [0, 0, 1, 0],
            [0, 0, 2, 0]])
solver.solve(_GRID)
# generator.solve_search(_GRID)

[[2, 2, 1, 1]
 [1, 1, 2, 2]
 [0, 2, 1, array([2, 1])]
 [0, 1, 2, array([2, 1])]]
array([0, 1])


TypeError: 

In [31]:
# generator = Generator()
# FLIP = np.array([0, 2, 1])
# grid = generator.generate_grid(4)
# print(grid)

int8=int
generator = Jenerator()
FLIP = array([0, 2, 1])
grid = generator.generate_grid(4)
print(grid)
print(jalid_grid(grid, verbose=True))

[[2, 2, 1, 1]
 [1, 1, 2, 2]
 [0, 0, 1, 0]
 [0, 0, 2, 0]]


TypeError: 

In [249]:
A = np.zeros((4, 4), dtype=np.int8)
A[0, 0] = 1
A[0, 1] = 1
# Find firs tinstance of 0
zeros = np.where(A == 0)
print(zeros)
tile = zeros[0][0], zeros[1][0]
tile

(array([0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3], dtype=int64), array([2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3], dtype=int64))


(0, 2)

In [201]:
A = [1, 2, 3 ,4]
while A:
    
    A.pop()
    print(A)

[1, 2, 3]
[1, 2]
[1]
[]


## Save and load

In [8]:
# SAVE AND LOAD configirations

# save = {str(i) : a for i, a in enumerate(configs)}  # Give them names 
configs = [[1, 0], A, B]  # Or just use a list
np.savez("duko_configs.npz", *configs)  # Save them

loaded = np.load("duko_configs.npz")
configs = loaded.values()
for kek in configs:
    print(kek)

[1 0]
[[1 1 2 2]
 [2 1 1 2]
 [2 2 1 1]
 [1 2 2 1]]
[[1 1 2 2]
 [2 2 1 1]
 [2 2 1 1]
 [1 2 2 1]]


In [9]:
# np.save('grid.npy', save)  # Save grid

# loaded = np.load('grid.npy')
# loaded